In [ ]:
from Mercados.funcoes import buscar_pao_de_acucar, buscar_extra, buscar_sonda, buscar_carrefour, buscar_st_marche

In [4]:
produto = "GRANOLA SALGADA MISS CROC MOSTARDA E MEL POUCH 120G"

In [5]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse

In [21]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

# Produto para buscar
produto = "BISCOITO MINI ARROZ CAMIL INTEGRAL 150G"  # Teste com algo que você sabe que existe no Sonda
produto_formatado = urllib.parse.quote(produto)

# URL do Sonda
url = f"https://www.sondadelivery.com.br/delivery/busca/{produto_formatado}"
headers = {
    "User-Agent": "Mozilla/5.0"
}

# Fazendo a requisição
response = requests.get(url, headers=headers)
response.encoding = 'utf-8'

# Verificando se a resposta foi bem-sucedida
if response.status_code == 200:
    print("✅ Requisição bem-sucedida!")
    soup = BeautifulSoup(response.text, 'html.parser')

else:
    print(f"❌ Erro na requisição: {response.status_code}")

# Exibindo um trecho do HTML retornado
soup


✅ Requisição bem-sucedida!



<!DOCTYPE html>

<html class="ieAll" lang="pt-br">
<head id="ctl00_Head1">
<!--
                <script src="/Scripts/js/plugins/picturefill.min.js"></script>
                <script src="/Scripts/js/vendor/modernizr-custom.min.js"></script>
                <script src="/Scripts/js/vendor/jquery/jquery.min.js"></script>
                Usado como lib geral
                <script src="/Scripts/js/vendor/bootstrap/bootstrap.min.js"></script>
                <script src="/Scripts/js/plugins/isMobile.min.js"></script>
                <script src="/Scripts/js/plugins/jquery.elevateZoom-3.0.8.min.js"></script>
                Usado para os plugins do Bootstrap
                <script src="/Scripts/js/plugins/icheck.js?v=2"></script>
                Usado para checkboxes e radios personalizados
                <script src="/Scripts/js/plugins/bootstrap-select.min.js"></script>
                Usado para select personalizado
                <script src="/Scripts/js/plugins/owl.carousel.js?v=

In [50]:
import requests
import urllib.parse
import re
from difflib import SequenceMatcher

# Função para encontrar o produto mais parecido usando análise mais detalhada
def encontrar_melhor_match(produto_pesquisado, produtos, chave_nome):
    if not produtos:
        return None  # Retorna None se não houver produtos

    # Definir palavras irrelevantes (não devem influenciar a correspondência)
    palavras_irrelevantes = {"de", "com", "c", "e", "o", "a", "do", "da", "zero", "light"}

    # Criar lista de palavras importantes na busca (removendo palavras irrelevantes)
    palavras_busca = set(produto_pesquisado.lower().split()) - palavras_irrelevantes

    melhor_produto = None
    maior_score = 0

    def similaridade(a, b):
        return SequenceMatcher(None, a, b).ratio()

    for produto in produtos:
        nome_produto = produto[chave_nome].lower()
        palavras_produto = set(nome_produto.split()) - palavras_irrelevantes

        # Similaridade entre strings (peso 40%)
        score_similaridade = similaridade(produto_pesquisado.lower(), nome_produto)

        # Cobertura de palavras (peso 40%) - Favorece produtos que contêm mais palavras da busca
        cobertura_palavras = len(palavras_busca.intersection(palavras_produto)) / len(palavras_busca)

        # Priorização de marca e sabor (peso 20%) - Favorece marcas e sabores específicos
        bonus_marca_sabor = 0
        for palavra in palavras_busca:
            if palavra in palavras_produto:
                bonus_marca_sabor += 0.1  # Aumenta a pontuação se a palavra for relevante

        # Score final = 40% similaridade + 40% cobertura + 20% bônus de marca/sabor
        score_final = (0.4 * score_similaridade) + (0.4 * cobertura_palavras) + (0.2 * bonus_marca_sabor)

        if score_final > maior_score:
            maior_score = score_final
            melhor_produto = produto

    return melhor_produto

# Função para buscar produtos no St Marche
def buscar_st_marche(produto):
    produto_formatado = urllib.parse.quote(produto)
    url = f"https://www.marche.com.br/produtos/autocomplete?query={produto_formatado}"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        produtos = []
        for item in data:
            nome = item.get("slug", "").replace("-", " ").title()
            preco = item.get("price", "Preço não disponível")
            link = f"https://www.marche.com.br/produtos/{item.get('slug', '')}"

            # Extrair imagem do campo "suggestion_view"
            suggestion_view = item.get("suggestion_view", "")
            match = re.search(r'https:\/\/mfresh\.s3\.amazonaws\.com\/[^\s"]+\.(?:jpg|png)', suggestion_view)
            imagem_url = match.group(0) if match else "Imagem não disponível"

            produtos.append({
                "nome": nome,
                "preco": preco,
                "imagem": imagem_url,
                "link": link
            })
        
        print(produtos)

        # Encontrar o melhor match
        melhor_produto = encontrar_melhor_match(produto, produtos, "nome")
        return melhor_produto, data

    return None

# Teste da função
produto_pesquisado = "BOLINHO ZERO SUAVIPAN CENOURA C/ CHOCOLATE 40G"
resultado, data = buscar_st_marche(produto_pesquisado)

# Exibir o melhor resultado encontrado
if resultado:
    print(resultado)
else:
    print("Nenhum produto encontrado no St Marche.")

data


[{'nome': 'Bolinho De Cenoura Com Recheio De Chocolate Light Suavipan 40G', 'preco': '6.9', 'imagem': 'https://mfresh.s3.amazonaws.com/uploads/product/sku/5658/image/mini_30887634-20ff-4ba3-b519-4f0605be922c.png', 'link': 'https://www.marche.com.br/produtos/bolinho-de-cenoura-com-recheio-de-chocolate-light-suavipan-40g'}, {'nome': 'Bolinho Suavipan Zero Chocolate 40G', 'preco': '6.9', 'imagem': 'https://mfresh.s3.amazonaws.com/uploads/product/sku/5688/image/mini_1f494e7a-a368-4557-97f4-faad61b0e18d.jpg', 'link': 'https://www.marche.com.br/produtos/bolinho-suavipan-zero-chocolate-40g'}, {'nome': 'Bolinho Zero Suavipan Red Velv Cream Cheese 40G', 'preco': '6.9', 'imagem': 'https://mfresh.s3.amazonaws.com/uploads/product/sku/26070/image/mini_c92a32c3-60e9-4fb1-a761-2b127b4e00b0.jpg', 'link': 'https://www.marche.com.br/produtos/bolinho-zero-suavipan-red-velv-cream-cheese-40g'}, {'nome': 'Bolinho Zero Suavipan De Churros De Doce De Leite 40G', 'preco': '6.9', 'imagem': 'https://mfresh.s3.am

[{'id': 6126,
  'sku_id': 5658,
  'is_master': True,
  'price': '6.9',
  'created_at': '2016-09-01T09:19:23.163-03:00',
  'updated_at': '2025-03-01T22:13:59.482-03:00',
  'is_default': True,
  'slug': 'bolinho-de-cenoura-com-recheio-de-chocolate-light-suavipan-40g',
  'promoted': False,
  'suggestion_view': '\n    <div class="row product-on-cart  product_group_5658 " data-role="js-product-on-cart" data-json="{&quot;id&quot;:null,&quot;full_name&quot;:&quot;Bolinho de Cenoura Com Recheio de Chocolate Light SUAVIPAN 40g&quot;,&quot;measure_type&quot;:&quot;un&quot;,&quot;quantity&quot;:0.0,&quot;mini_image&quot;:&quot;https://mfresh.s3.amazonaws.com/uploads/product/sku/5658/image/mini_30887634-20ff-4ba3-b519-4f0605be922c.png&quot;,&quot;category&quot;:&quot;Bolos Industrializados&quot;,&quot;slug&quot;:&quot;bolinho-de-cenoura-com-recheio-de-chocolate-light-suavipan-40g&quot;,&quot;group_id&quot;:5658,&quot;product_id&quot;:6126,&quot;total_on_cart&quot;:0.0,&quot;name&quot;:&quot;Bolinh

In [3]:
pip install playwright

Note: you may need to restart the kernel to use updated packages.


In [6]:
from browser_use import Browser

# Iniciar o navegador
browser = Browser()

# Acessar um site
browser.open("https://www.google.com")

# Digitar algo na barra de pesquisa e pressionar ENTER
browser.type("input[name=q]", "Browser-Use Python")
browser.press("Enter")

# Capturar o título da página
title = browser.get_title()
print(f"Título da página: {title}")

# Fechar o navegador
browser.quit()


AttributeError: 'Browser' object has no attribute 'open'